#Управление контекстом

В python управление контекстом осуществляется при помощи контекстных менеджеров.

**Контекстный менеджер** - это объект, который выполняет за вас рутинную работу, когда вы используете определённые ресурсы. 

Менеджер контекста задаёт временный контекст и ликвидирует его после выполнения операций.

#Что такое контекстный менеджер

Контекстные менеджеры упрощают работу с ресурсами. Они очень эффективны в их выделении и освобождении. По своей сути контекстные менеджеры из себя представляют блок кода, который будет гарантировать правльное выделение и освобожение используемых ресурсов.

Самой популярной конструкцией является `with`. 

Если вы хоть раз работали с файлами в python, скорее всего, вы использовали именно эту конструкцию.

Рассмотрим пример, на котором будут показаны преимущества работы с контекстным менеджером. 

Задача состоит в следующем: записать в файл *file.txt* строку *hello*.

**Самый простой** способ ее решить: 

1.   Воспользоваться функцией `open()` для открытия файла.
2.   Записать в него необходимые данные через функцию `write()`.
3.   Освободить выделенный на это действие ресурс, вызвав функцию `close()`.



In [ ]:
f = open(‘file.txt’, ‘w’)
f.write(‘hello’)
f.close()

Но **это плохое решение**, если в процессе работы с файлом (запись, чтение), произошло исключение, то функция `close()` не будет вызвана, что **влечет за собой возможную потерю ресурсов**. 

In [ ]:
f = open(‘file.txt’, ‘w’)
f.write(‘hello’)
raise ValueError('STOP') 
f.close()

Для предотвращения возможных потерь необходимо грамотно обработать исключения :

In [ ]:
f = open('file.txt', 'w')
try:
    f.write('hello')
except:
    print('Some error!')
finally:
    f.close()

Таким образом, соединив первое решение и грамотную обработку исключений мы получем, что для решения данной задачи понадобилось написать 7 строчек кода, в которых можно ошибиться или просто забыть описать какой-то важный момент. Таким образом мы рискуем "потерять" выделенные ресурсы!

И это только при простом открытии одного файла. Если же вам понадобится открывать файлы или базы данных в разных частях вашей программы, вам придётся каждый раз грамотно обрабатывать работу с ресурсами. Это будет в явном виде вызывать дублирование кода, что повышает риск ошибки.



Для решения задачи, избегая выше указанных проблем, можно использовать контекстный менеджер `with()` :

In [ ]:
with open('file.txt', 'w') as f:
     f.write('hello')

Основное преимущество использования with - это гарантия закрытия файла вне зависимости от того, как будет завершён вложенный код.

Таким образом, мы написали всего 2 строчки кода, в которых сложнее ошибиться. 

Такая конструкция позволяет захватить ресурс (в данном случае файл), выполнить нужный набор операций (запись данных), а перед выходом – освободить ресурс.

**Данное решение является самым эффективным**.

## Более детальный пример

In [15]:
#класс который будет выступать в виде произвольного ресурса
class Resource:
  def __init__(self):
    self.opened = False

  def open(self, *args):
    print(f'Открыли ресурс с аргументами {args}')
    self.opened = True

  def close(self):
    print('Закрыли ресурс')
    self.opened = False

  def __del__(self):
    if self.opened:
      print("Произошла утечка!")

  def action(self):
    print("Что-то делаю с ресурсом")

In [16]:
#Если у нас всё пошло по плану и мы хорошие программисты
resource = Resource()
resource.open(1, 2, 3)
resource.action()
resource.close()

Открыли ресурс с аргументами (1, 2, 3)
Что-то делаю с ресурсом
Закрыли ресурс


In [18]:
#Допустим, что мы забыли закрыть наш ресурс, что произойдёт?
resource = Resource()
resource.open(1, 2, 3)
resource.action()
#resource.close()

Открыли ресурс с аргументами (1, 2, 3)
Что-то делаю с ресурсом
Произошла утечка!


In [19]:
#Допустим, что мы грамотно описали работу с нашим ресурсом
#Но по какой-то причине получили исключение при работе с ним
resource = Resource()
resource.open(1, 2, 3)
resource.action()
raise ValueError('Что-то пошло не так')
resource.close()

Открыли ресурс с аргументами (1, 2, 3)
Что-то делаю с ресурсом


ValueError: ignored

Произошла утечка!


В этих двух случаях, как видно по выводу, ресурсы не были грамотно закрыты.
Это плохо сказывается на нашей программе.

Представим, что нам надо как-то избежать данных проблем. 

Для этого можно использовать контрукцию `try, except, finally`:

In [22]:
resourse = None

try:
  resource = Resource()
  resource.open(1, 2, 3)
  resource.action()
except:
  raise
finally:
  if resource:
    resource.close()

Открыли ресурс с аргументами (1, 2, 3)
Что-то делаю с ресурсом
Закрыли ресурс


In [25]:
#Что если мы получим исключение?
resourse = None

try:
  resource = Resource()
  resource.open(1, 2, 3)
  resource.action()
  raise ValueError()
except:
  raise
finally:
  if resource:
    resource.close()

Открыли ресурс с аргументами (1, 2, 3)
Что-то делаю с ресурсом
Закрыли ресурс


ValueError: ignored

Даже при исключении ресурс закрывается и мы не получаем утечку.

#Как работает менеджер контекста `with`

Синтаксис оператора `with`

In [ ]:
with EXPRESSION as TARGET:
    SUITE



Семантический эквивалентен:


In [ ]:
manager = (EXPRESSION)
enter = type(manager).__enter__
exit = type(manager).__exit__
value = enter(manager)
hit_except = False

try:
    TARGET = value
    SUITE
except:
    hit_except = True
    if not exit(manager, *sys.exc_info()):
        raise
finally:
    if not hit_except:
        exit(manager, None, None, None)

Выражение `EXPRESSION`, непосредственно следующее за ключевым словом `with` является "**выражением контекста**", так как это выражение обеспечивает основной ключ к среде выполнения, которую менеджер контекста устанавливает для продолжительности тела выражения.



1.   Выражение контекста (выражение, указанное в `EXPRESSION`) оценивается для получения менеджера контекста.

2.   Менеджер контекста загружает метод `__enter__()` для последующего использования.
3.   Менеджер контекста загружает метод `__exit__()` для последующего использования.
4.   Менеджер контекста вызывает метод `__enter__()`.
5.   Если `TARGET` была включена в оператор `with`, то ей присваивается возвращаемое значение из метода `__enter__()`.
*Обратите внимание, что оператор `with` гарантирует, что если метод `__enter__()` возвращается без ошибки, то всегда будет вызываться метод `__exit__()`. Таким образом, если ошибка возникает во время присваивания значения через оператор `as`, то она будет обрабатываться так же, как и ошибка, возникающая внутри `with`.*


6.   Последовательность команд выполнена.
7.   Вызван метод `__exit__()`. Если исключение вызвало выход из последовательности команд, то его тип `exc_type`, значение `exc_val` и информация о трассировке `exc_tb` передаются в качестве аргументов `__exit__()`. В противном случае предоставляется три аргумента `None`.

Если последовательность команд была завершена из-за исключения, а возвращаемое значение из метода `__exit__()` было `False`, то исключение вызывается повторно. 

Если возвращаемое значение было `True`, то исключение подавляется и выполнение продолжается с оператора, следующего за оператором `with`.

Если последовательность команд была завершена по любой причине, кроме исключения, то возвращаемое значение из `__exit__()` игнорируется, и выполнение продолжается.

При наличии нескольких контекстных менеджеров, они обрабатываются так, как если бы несколько операторов `with` были вложенными :

In [ ]:
with A() as a, B() as b:
    SUITE

# Эквивалентно

with A() as a:
    with B() as b:
        SUITE


# Протокол менеджера контекста

Создавать менеджеры контекста можно, чтобы самостоятельно управлять некоторыми ресурсами. **Одним из** способов создания является реализация методов для протокола менеджера контекста. Можете представить это себе как утиную типизацию  —  мы просто определим магические методы `__enter__` и `__exit__` без формального согласования протокола или реализации интерфейса. 

Следующий код демонстрирует эту концепцию:

In [30]:
class ContextManagerExample:
  def __init__(self):
    print("Менеджер создан")

  def __enter__(self):
    print("Начать управление контекстом")
 
  def __exit__(self, exc_type, exc_val, exc_tb):
    print("Завершить управление контекстом")

with ContextManagerExample():
  print("Запуск операций в операторе with")

Менеджер создан
Начать управление контекстом
Запуск операций в операторе with
Завершить управление контекстом


Как показано выше, мы просто определили класс, в котором реализованы методы `__enter__` и `__exit__`, **способные управлять контекстом за нас**. С синтаксической точки зрения, мы можем использовать этот класс в операторе `with`, что и было сделано.

Выведенный текст показывает нам порядок, в котором эти операции хорошо координируются. В частности, созданный экземпляр вызовет метод `__enter__` для запуска контекста, затем мы сами выполняем операции, и, наконец, менеджер контекста выйдет из управления, вызвав метод `__exit__`.

#Модуль contextlib

Вы обнаружите, что самостоятельная реализация специальных методов `__enter__` и `__exit__` для создания менеджера контекста может оказаться утомительной. С модулем **contextlib** в стандартной библиотеке Python намного проще управлять контекстом.

Python позволяет нам создать контекстный менеджер, используя функцию модуля **contextlib** под названием `contextmanager` в качестве **декоратора**. 

Контекстный менеджер который открывает и закрывает файл после проделанной в нем работе:

In [35]:
from contextlib import contextmanager
 
@contextmanager
def file_open(path):
    try:
        f_obj = open(path, 'w')
        yield f_obj
    except OSError:
        print("We had an error!")
    finally:
        print('Closing file')
        f_obj.close()
 
 
with file_open('test.txt') as fobj:
  fobj.write('Testing context managers')

Closing file


Здесь мы просто импортируем `contextmanager` из `contextlib` и декорируем нашу функцию `file_open` с ним. Это позволяет нам вызвать `file_open` используя оператор `with`. В нашей функции мы открываем файл, отдаем его, чтобы функция `calling` могла использовать его. После того, как оператор закончит, контроль возвращается обратно к функции `file_open`, которая продолжает следовать по коду за вызываемым оператором. Это приводит оператор `finally` к исполнению, благодаря которому и закрывается файл. Если возникла ошибка `OSError` во время работы с файлом, она будет выявлена и оператор `finally` закроет обработчик файлов несмотря на это.

## contextlib.closing()

Модуль `contextlib` содержит несколько полезных утилит. 

Первая – это класс `closing`, который закроет объект по завершению определенного блока кода. В документации Python есть пример кода, похожий на следующий:

In [33]:
from contextlib import contextmanager
 
@contextmanager
def closing(db):
    try:
        yield db.conn()
    finally:
        db.close()

В целом, мы создаем **закрывающую функцию**, которая завернута в контекстный менеджер. Это эквивалент того, что делает **класс closing**. 

Но есть небольшая разница: вместо декоратора, мы можем использовать класс в нашем операторе `with` :

In [34]:
from contextlib import closing
from urllib.request import urlopen
 
with closing(urlopen('http://www.google.com')) as webpage:
    for line in webpage:
        # обрабатываем строку...
        pass

В данном примере мы открыли страницу URL, но обернули её в наш класс `closing`. Это приведет к закрытию дескриптора веб-страницы, сразу после выхода из блока кода оператора `with`.

## contextlib.suppress(*exceptions)

Еще один полезный инструмент — класс `suppress`. Идея в том, что данная утилита контекстного менеджера **может подавлять любое количество исключений**. 

Скажем, нам нужно проигнорировать исключение `FileNotFoundError`. Если прописать следующий контекстный менеджер, то это не сработает:

In [36]:
with open('fauxfile.txt') as fobj:
  for line in fobj:
    print(line)

FileNotFoundError: ignored

Как видим, этот контекстный менеджер **не выполняет обработку данного исключения**. Если вам нужно проигнорировать эту ошибку, лучше напишите следующий код:

In [37]:
from contextlib import suppress
 
with suppress(FileNotFoundError):
  with open('fauxfile.txt') as fobj:
    for line in fobj:
      print(line)

Здесь мы **импортируем `suppress`** и передаем его исключению **`FileNotFoundError`**. 
Если вы запустите этот код, вы увидите, что ничего не происходит, так как файл не существует, но и **ошибка не возникает**.

## ExitStack

**ExitStack** – это контекстный менеджер, который позволит вам легко комбинировать другие контекстные менеджеры, а также функции очистки.

Простой пример из документации Python:

In [41]:
from contextlib import ExitStack

with ExitStack as stack:
  file_objects = [stack.enter_context(open(fname)) for filename in filenames]

Данный код создает серию контекстных менеджеров внутри списка. `ExitStack` поддерживает стек регистрируемых колбеков, которые **вызываются в обратом порядке** когда экземпляр закрыт, что и происходит, когда мы выходим из оператора `with`.

## Реентерабельные контекстные менеджеры

Большая часть создаваемых вами контекстных менеджеров может быть написана только для использования с оператором `with` для **одноразового применения**.

In [42]:
from contextlib import contextmanager
 
@contextmanager
def single():
    print('Yielding')
    yield
    print('Exiting context manager')
 
context = single()
with context:
    pass

Yielding
Exiting context manager


In [43]:
with context:
    pass

AttributeError: ignored

Здесь мы создали **экземпляр контекстного менеджера** и пытаемся запустить его дважды с оператором **`with`**. Второй запуск приводит к ошибке `RuntimeError`. 

Но что делать, если нам необходимо, чтобы контекстный менеджер запускался дважды? Для этой цели нам и нужен **реентрабельный контекстный менеджер**. 

Рассмотрим менеджер redirect_stdout, который будем использовать в данном примере.

### contextlib.redirect_stdout



Библиотека contextlib содержит несколько замечательных инструментов для перенаправления stdout и stderr. 

До того, как эти инструменты появились, и когда вам нужно перенаправить stdout, вам нужно сделать что-то на подобии этого:

In [ ]:
path = '/path/to/text.txt'
 
with open(path, 'w') as fobj:
    sys.stdout = fobj
    help(sum)

С модулем **`contextlib`** вы можете сделать следующее:

In [ ]:
from contextlib import redirect_stdout
 
path = '/path/to/text.txt'
with open(path, 'w') as fobj:
    with redirect_stdout(fobj):
        help(redirect_stdout)

В обоих примерах мы перенаправили `stdout` к файлу. Когда мы вызываем справку `Python`, вместо вывода в `stdout`, она сохраняется **непосредственно в файле**. Вы также можете перенаправить `stdout` в какой-нибудь буфер.

После ознакомления с redirect_stdout, вернёмся к **реентрабельным контекстным менеджерам**.

In [44]:
from contextlib import redirect_stdout
from io import StringIO
 
stream = StringIO()
write_to_stream = redirect_stdout(stream)
with write_to_stream:
    print('Write something to the stream')
    with write_to_stream:
        print('Write something else to stream')
 
print(stream.getvalue())

Write something to the stream
Write something else to stream



Здесь мы создали вложенные контекстные менеджеры, которые оба пишут в **`StringIO`**, который является **текстовым потоком в памяти**. 

Причина, по которой это работает, а не приводит к ошибке `RuntimeError`, как было ранее в том, что **`redirect_stdout` является реентрабельным** **и** **позволяет нам вызывать его дважды**.